Prompts
A prompt for a language model is a set of instruction or input provided by a user to guid the model's response.
Prompts can be defined with **PromptTemplate** and **ChatPromptTemplate**

In [1]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template("Tell me a {adjective} joke about {topic}?")
result = prompt_template.format(adjective="funny", topic="chicken")
result

'Human: Tell me a funny joke about chicken?'

With **ChatPromptTemplate**, it is required to provide an additional parameter called *role*. For example, OpanAi Chat Completion API have System, AI, or Human role.

In [2]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Your name is {name}"),
        ("human", "Hello, how are you?"),
        ("assistant", "I am fine, thank you!"),
        ("human", "{user_input}"),
    ]
)

messages = chat_template.format_messages(name="Alice", user_input="What is the weather like today?")

messages

[SystemMessage(content='You are a helpful assistant. Your name is Alice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='I am fine, thank you!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is the weather like today?', additional_kwargs={}, response_metadata={})]

**Chat Models**

In [3]:
from langchain_openai import AzureChatOpenAI
import os
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, SystemMessage

load_dotenv()

AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")

llm = AzureChatOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    deployment_name=AZURE_OPENAI_DEPLOYMENT_NAME,
    api_version=AZURE_OPENAI_API_VERSION,
    api_key=AZURE_OPENAI_KEY
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="What is the capital of France?")
]

response = llm.invoke(messages)
response

AIMessage(content='The capital of France is **Paris**.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 24, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b54fe76834', 'id': 'chatcmpl-Cl5CBeXWsSylLetGAkW8cS5jDDLXh', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'sa

**Completion**

In [4]:
from openai import AzureOpenAI

client = AzureOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_KEY,
    api_version=AZURE_OPENAI_API_VERSION
)

response = client.chat.completions.create(
    model=AZURE_OPENAI_DEPLOYMENT_NAME,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Does Azure OpenAI support customer managed keys?"},
        {"role": "assistant", "content": "Yes, customer managed keys are supported by Azure OpenAI."},
        {"role": "user", "content": "Do other Azure services support this too?"},
    ]
)

response.choices[0].message.content

'Yes, many Azure services support customer-managed keys (CMK). Customer-managed keys give you full control over encryption keys, allowing you to create, manage, and audit their use. These keys are typically stored in **Azure Key Vault** or **Azure Managed HSM** (Hardware Security Module). \n\nHere are some commonly used Azure services that support customer-managed keys:\n\n### 1. **Azure Storage Services**\n   - **Azure Blob Storage**: You can use CMK to encrypt and protect your blob storage data.\n   - **Azure Files**: Supports CMK for encrypting Azure file shares.\n   - **Azure Queue Storage**: Data can be encrypted using CMK.\n\n### 2. **Azure Disk Encryption**\n   - Azure Disk Encryption enables you to encrypt virtual machine (VM) disks with CMK. You can use your keys stored in Azure Key Vault for this purpose.\n\n### 3. **Azure SQL Database and Managed Instance**\n   - Both **Azure SQL Database** and **Azure SQL Managed Instance** support the use of CMK for Transparent Data Encryp

In [5]:
response.choices

[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Yes, many Azure services support customer-managed keys (CMK). Customer-managed keys give you full control over encryption keys, allowing you to create, manage, and audit their use. These keys are typically stored in **Azure Key Vault** or **Azure Managed HSM** (Hardware Security Module). \n\nHere are some commonly used Azure services that support customer-managed keys:\n\n### 1. **Azure Storage Services**\n   - **Azure Blob Storage**: You can use CMK to encrypt and protect your blob storage data.\n   - **Azure Files**: Supports CMK for encrypting Azure file shares.\n   - **Azure Queue Storage**: Data can be encrypted using CMK.\n\n### 2. **Azure Disk Encryption**\n   - Azure Disk Encryption enables you to encrypt virtual machine (VM) disks with CMK. You can use your keys stored in Azure Key Vault for this purpose.\n\n### 3. **Azure SQL Database and Managed Instance**\n   - Both **Azure SQL Data

**Output Parsers**
- Language models output text but many times we may want to get more structured information thant just text
- An output parser must implement 2 methods:
  - *Get format instructions*: A method which returns a string containing instructions for how the output of a language model should be formatted
  - *Parse*: A method which takes in a string(assumed to be the response from a language model) and parses it into some structure
- LangChain has several built-int output parsers:
  - List parser
  - Datetime parser
  - Enum parser
  - ConvoOutputParser

**Chains**
- Using an LLM in a single step is fine for simple applications but more complex applications require chaining LLMs - either with each other LLM or with other components.
- There are several chain types:
  - LLM chain: Is the most common type of chaining in LLM application
  - Router chain: Route to different operations based on user input
  - Sequential chain: Combine multiple LLM chain into a pipeline
  - Transformation chain: Transform inputs with LLM

In [6]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    deployment_name=AZURE_OPENAI_DEPLOYMENT_NAME,
    api_version=AZURE_OPENAI_API_VERSION,
    api_key=AZURE_OPENAI_KEY,
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

# Use LLM Chain
prompt_template = """Tell me a joke about {topic}?
Answer:
"""

llm_chain = ChatPromptTemplate.from_template(prompt_template) | llm
result = llm_chain.invoke({"topic": "robot"})
result.content

"Sure! Here's a robot joke for you:\n\nWhy did the robot go on a diet?  \nBecause he had a byte problem! 🤖"

**Tools**
- Tools are interfaces that an agent, chain, or LLM can use to interact with the world
- They combine a few things:
  - The name of the tool
  - A description of what the tool does
  - JSON schema of the inputs to the tool
  - The function to call
  - Whether the result of the tool should be returned directly to the user
- Tools can be used to:
  - Search the web
  - Query databases
  - Perform calculations
  - Call APIs
  - Execute custom functions

**Creating a Custom Tool**

In [7]:
from langchain.tools import tool

@tool
def multiply(a: float, b: float) -> float:
    """Multiply two numbers together."""
    return a * b

@tool
def add(a: float, b: float) -> float:
    """Add two numbers together."""
    return a + b

# Test the tools
print(f"Tool name: {multiply.name}")
print(f"Tool description: {multiply.description}")
print(f"Tool args: {multiply.args}")
print(f"Result: {multiply.invoke({'a': 5, 'b': 3})}")
print()
print(f"Add result: {add.invoke({'a': 10, 'b': 7})}")

Tool name: multiply
Tool description: Multiply two numbers together.
Tool args: {'a': {'title': 'A', 'type': 'number'}, 'b': {'title': 'B', 'type': 'number'}}
Result: 15.0

Add result: 17.0


**Using Tools with LLM (Function Calling)**
- Modern LLMs can decide when and how to use tools
- The LLM analyzes the user's request and determines which tool to call
- This is also known as "function calling" or "tool calling"

In [8]:
from langchain_openai import AzureChatOpenAI
from langchain.tools import tool

# Define tools
@tool
def get_weather(city: str) -> str:
    """Get the current weather for a given city."""
    # In a real scenario, this would call a weather API
    return f"The weather in {city} is sunny with a temperature of 72°F"

@tool
def calculate_age(birth_year: int) -> int:
    """Calculate age based on birth year."""
    from datetime import datetime
    current_year = datetime.now().year
    return current_year - birth_year

# Bind tools to the LLM
tools = [get_weather, calculate_age]
llm_with_tools = llm.bind_tools(tools)

# The LLM will decide which tool to use based on the user's question
response = llm_with_tools.invoke("What's the weather like in Paris?")
print(response)
print()
print("Tool calls:", response.tool_calls)

content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 75, 'total_tokens': 90, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b54fe76834', 'id': 'chatcmpl-Cl5CRV5iR9PiesIsFiJt3eXG6eeWY', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'tool_calls', 'logprobs': None, 'content_filter_results': {}} id='lc_run--019b064f-9ba7-7850-9424-ecf420809ceb-0' tool_calls=[{'name': 'get_wea

**Executing Tool Calls**
- When the LLM decides to use a tool, it returns a tool call
- We need to execute the tool and return the result back to the LLM

In [9]:
from langchain_core.messages import HumanMessage, ToolMessage

# Get the LLM's response with tool call
messages = [HumanMessage(content="What's the weather in Tokyo?")]
ai_msg = llm_with_tools.invoke(messages)

# Check if the LLM wants to use a tool
if ai_msg.tool_calls:
    # Execute the tool
    for tool_call in ai_msg.tool_calls:
        tool_name = tool_call["name"]
        tool_args = tool_call["args"]
        
        # Find and execute the matching tool
        selected_tool = {tool.name: tool for tool in tools}[tool_name]
        tool_output = selected_tool.invoke(tool_args)
        
        print(f"Tool: {tool_name}")
        print(f"Arguments: {tool_args}")
        print(f"Result: {tool_output}")
        
        # Continue conversation with tool result
        messages.append(ai_msg)
        messages.append(ToolMessage(content=str(tool_output), tool_call_id=tool_call["id"]))
        
        # Get final response from LLM
        final_response = llm_with_tools.invoke(messages)
        print(f"\nFinal Response: {final_response.content}")

Tool: get_weather
Arguments: {'city': 'Tokyo'}
Result: The weather in Tokyo is sunny with a temperature of 72°F

Final Response: The weather in Tokyo is sunny with a temperature of 72°F.

Final Response: The weather in Tokyo is sunny with a temperature of 72°F.


**Built-in Tools**
- LangChain provides many pre-built tools for common tasks
- Examples include: Wikipedia search, DuckDuckGo search, Python REPL, and more

In [10]:
# Example: Using Wikipedia tool (requires: pip install wikipedia)
# Uncomment to use:
# from langchain_community.tools import WikipediaQueryRun
# from langchain_community.utilities import WikipediaAPIWrapper

# wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
# result = wikipedia.invoke("LangChain")
# print(result)

# For now, let's create a simple search tool
@tool
def search_database(query: str) -> str:
    """Search a database for information."""
    # Mock implementation
    database = {
        "python": "Python is a high-level programming language",
        "langchain": "LangChain is a framework for developing LLM applications",
        "azure": "Azure is Microsoft's cloud computing platform"
    }
    return database.get(query.lower(), "No information found")

result = search_database.invoke({"query": "python"})
print(result)

Python is a high-level programming language
